In [1]:
import pandas as pd
import numpy as np

In [2]:
#import data
suffix_no_nuclear = 'no-nuclear_no_coal_renewables'
suffix_normal = 'normal'

gen = pd.read_csv(f'./good_model_inputs/inputs_gen_{suffix_no_nuclear}.csv')
load = pd.read_csv(f'./good_model_inputs/inputs_load_no-nuclear.csv')
rencf = pd.read_csv(f'./good_model_inputs/inputs_renewableCF.csv')
trans = pd.read_csv(f'./good_model_inputs/inputs_trans_no-nuclear.csv')

In [3]:
#create a list of our regions
regions = set(load['r'])

In [5]:
#calculate regions where generation > load
for reg in regions:
    gn = np.zeros((8759,))
    gn += gen.loc[(gen['RegionName'] == reg).values * (gen['FuelType'] != 'Solar').values * (gen['FuelType'] != 'Wind').values * (gen['FuelType'] != 'wind_generator').values * (gen['FuelType'] != 'solar_generator').values, 'Capacity'].values.sum()
    gn += (gen.loc[(gen['RegionName'] == reg).values * (gen['FuelType'] == 'Solar').values, 'Capacity'].values.sum() * rencf.loc[rencf['r'] == reg, 'solarCF'].values).sum()
    gn += (gen.loc[(gen['RegionName'] == reg).values * (gen['FuelType'] == 'solar_generator').values, 'Capacity'].values.sum() * rencf.loc[rencf['r'] == reg, 'solarCF'].values).sum()
    gn += (gen.loc[(gen['RegionName'] == reg).values * (gen['FuelType'] == 'Wind').values, 'Capacity'].values.sum() * rencf.loc[rencf['r'] == reg, 'windCF'].values).sum()
    gn += (gen.loc[(gen['RegionName'] == reg).values * (gen['FuelType'] == 'wind_generator').values, 'Capacity'].values.sum() * rencf.loc[rencf['r'] == reg, 'windCF'].values).sum()

    if (load.loc[load['r'] == reg, 'demandLoad'] > gn).values.sum() > 0:
        print(f'Generation cannot match load in region {reg}')
        print(max(load.loc[load['r'] == reg, 'demandLoad'] - gn))
    

In [32]:
#find the total generation for the region where generation > load

region = 'ERC_REST'
t = 5080

print( 'Nuclear generation ' + str(gen.loc[gen['RegionName'] == region].loc[gen['FuelType'] == 'Nuclear','Capacity'].sum()))

print( 'Total generation ' + str(gen.loc[gen['RegionName'] == region,'Capacity'].sum()))

#look at the transmission into the region where generation > load

print( 'Transmission into the region ' + str(trans.loc[(trans['r2'] == region).values * (trans['transCap'] > 1).values, 'transCap'].sum()*.972))

print( 'Transmission out of the region ' + str(trans.loc[(trans['r1'] == region).values * (trans['transCap'] > 1).values, 'transCap'].sum()))

#find the total amount of energy needed to/from that region if all used

print( f'Maximum net energy left after use from the region at time {t} ' + 
    str(trans.loc[(trans['r2'] == region).values, 'transCap'].sum()*.972 + 
       gen.loc[gen['RegionName'] == region,'Capacity'].sum() - 
        trans.loc[(trans['r1'] == region).values, 'transCap'].sum() - 
       load.loc[(load['r'] == 'NY_Z_GI')].loc[(load['t'] == t)]['demandLoad'].values))

Nuclear generation 0.0
Total generation 451309.1
Transmission into the region 11916.72
Transmission out of the region 5529
Maximum net energy left after use from the region at time 5080 [455072.832]


In [33]:
#find what time stamps are associated with demand > load 
#(we will go back and manually change our load based on this, assuming some of the region blacks out)
load.loc[load['r'] == region].loc[load['t'].isin(range(5001,6000))].loc[load['demandLoad'] >= gen.loc[gen['RegionName'] == region,'Capacity'].sum()]

,Unnamed: 0,r,t,demandLoad
